<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for accessing Forecastingdata forecasting and feature extraction tools using python

in Forecastingdata, some forecasting and feature calculation tools are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook shows an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
runs directly on python. Part2 is an example of how to use the tools directly from python.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [1]:
#@title
%load_ext rpy2.ipython

Clone the repository, for reproducibility. No need to clone everytime if you have already have the folder!.

In [2]:
!git clone https://github.com/pmontman/TSForecasting

Cloning into 'TSForecasting'...
remote: Enumerating objects: 377, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 377 (delta 36), reused 30 (delta 30), pack-reused 333
Receiving objects: 100% (377/377), 274.66 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (191/191), done.


In [3]:
%cd TSForecasting/
!git pull
%cd ..

/Users/jerryobex/proj/TSForecasting-wrapper/TSForecasting


/Users/jerryobex/proj/TSForecasting-wrapper/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Already up to date.
/Users/jerryobex/proj/TSForecasting-wrapper


We now prepare the R environment for processing: installing and loading the required packages

```sh
cat << EOM > ~/.R/Makevars                                    
BREWDIR:=$(shell brew --prefix)
CPPFLAGS=-I$(BREWDIR)/include
LDFLAGS=-L$(BREWDIR)/lib
EOM

 brew install r harfbuzz fribidi
 brew install libgit2
 brew install freetype libpng libtiff zlib libjpeg-turbo
```


In [8]:
%%R
install.packages("devtools")
library("devtools")
install.packages("Rcatch22")

* installing *source* package ‘devtools’ ...
** package ‘devtools’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (devtools)
* installing *source* package ‘Rcatch22’ ...
** package ‘Rcatch22’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using C++ compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using SDK: ‘MacOSX14.2.sdk’


clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c CO_AutoCorr.c -o CO_AutoCorr.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c DN_HistogramMode_10.c -o DN_HistogramMode_10.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c DN_HistogramMode_5.c -o DN_HistogramMode_5.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c DN_Mean.c -o DN_Mean.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG -I../inst/include -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/o

installing to /opt/homebrew/lib/R/4.3/site-library/00LOCK-Rcatch22/00new/Rcatch22/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (Rcatch22)


Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/devtools_2.4.5.tar.gz'
Content type 'application/x-gzip' length 374718 bytes (365 KB)
downloaded 365 KB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/Rtmphiu8f4/downloaded_packages’
Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/Rcatch22_0.2.1.tar.gz'
Content type 'application/x-gzip' length 191609 bytes (187 KB)
downloaded 187 KB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/Rtmphiu8f4/downloaded_packages’


In [9]:
%%R
install.packages("tidyverse")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
install.packages("smooth")

* installing *source* package ‘tidyverse’ ...
** package ‘tidyverse’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (tidyverse)
* installing *source* package ‘tsibble’ ...
** package ‘tsibble’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed pack

clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c calcBATS.cpp -o calcBATS.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c calcTBATS.cpp -o calcTBATS.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c etsTargetFunction.cpp -o etsTargetFunction.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArm

installing to /opt/homebrew/lib/R/4.3/site-library/00LOCK-forecast/00new/forecast/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (forecast)
* installing *source* package ‘tsfeatures’ ...
** package ‘tsfeatures’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed packag

clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c RcppExports.cpp -o RcppExports.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c adamGeneral.cpp -o adamGeneral.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArmadillo/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c adamRefitter.cpp -o adamRefitter.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I'/opt/homebrew/lib/R/4.3/site-library/RcppArm

installing to /opt/homebrew/lib/R/4.3/site-library/00LOCK-smooth/00new/smooth/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (smooth)


Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/tidyverse_2.0.0.tar.gz'
Content type 'application/x-gzip' length 704618 bytes (688 KB)
downloaded 688 KB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/Rtmphiu8f4/downloaded_packages’
Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/tsibble_1.1.4.tar.gz'
Content type 'application/x-gzip' length 1552269 bytes (1.5 MB)
downloaded 1.5 MB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/Rtmphiu8f4/downloaded_packages’
Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/forecast_8.21.1.tar.gz'
Content type 'application/x-gzip' length 591793 bytes (577 KB)
downloaded 577 KB



Install glmnet package for linear models

In [10]:
%%R
install.packages('glmnet')

* installing *source* package ‘glmnet’ ...
** package ‘glmnet’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using Fortran compiler: ‘GNU Fortran (Homebrew GCC 13.2.0) 13.2.0’
using C++ compiler: ‘Apple clang version 15.0.0 (clang-1500.1.0.2.5)’
using C++17
using SDK: ‘MacOSX14.2.sdk’


clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c RcppExports.cpp -o RcppExports.o
/opt/homebrew/opt/gcc/bin/gfortran  -fPIC  -g -O2  -c coxnet5dpclean.f -o coxnet5dpclean.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c elnet_exp.cpp -o elnet_exp.o


In file included from elnet_exp.cpp:2:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigen.h:25:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigenForward.h:28:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Dense:1:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Core:367:
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/util/Meta.h:320:25: warning: 'result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' is deprecated [-Wdeprecated-declarations]
  typedef typename std::result_of<T>::type type1;
                        ^
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/CwiseBinaryOp.h:33:20: note: in instantiation of template class 'Eigen::internal::result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' requested here
  typedef typename result_of<


clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c glmnet_init.cpp -o glmnet_init.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include   -Iglmnetpp/include -Iglmnetpp/src -Iglmnetpp/test -DEIGEN_PERMANENTLY_DISABLE_STUPID_WARNINGS -fPIC  -g -O2  -c internal.cpp -o internal.o
clang -I"/opt/homebrew/Cellar/r/4.3.2/lib/R/include" -DNDEBUG  -I'/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include' -I'/opt/homebrew/lib/R/4.3/site-library/Rcpp/include' -I/opt/homebrew/include    -fPIC  -g -O2  -c pb.c -o pb.o
clang++ -std=gnu++17 -I"/opt/homebrew/Cellar/r/4

In file included from wls_exp.cpp:2:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigen.h:25:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/RcppEigenForward.h:28:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Dense:1:
In file included from /opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/Core:367:
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/util/Meta.h:320:25: warning: 'result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' is deprecated [-Wdeprecated-declarations]
  typedef typename std::result_of<T>::type type1;
                        ^
/opt/homebrew/lib/R/4.3/site-library/RcppEigen/include/Eigen/src/Core/CwiseBinaryOp.h:33:20: note: in instantiation of template class 'Eigen::internal::result_of<Eigen::internal::scalar_sum_op<double> (const double &, const double &)>' requested here
  typedef typename result_of<
  

clang++ -std=gnu++17 -dynamiclib -Wl,-headerpad_max_install_names -undefined dynamic_lookup -L/opt/homebrew/Cellar/r/4.3.2/lib/R/lib -L/opt/homebrew/lib -o glmnet.so RcppExports.o coxnet5dpclean.o elnet_exp.o glmnet_init.o internal.o pb.o pb_exp.o wls_exp.o -L/opt/homebrew/opt/gcc/lib/gcc/current/gcc/aarch64-apple-darwin23/13 -L/opt/homebrew/opt/gcc/lib/gcc/current/gcc -L/opt/homebrew/opt/gcc/lib/gcc/current -lgfortran -lemutls_w -lquadmath -L/opt/homebrew/Cellar/r/4.3.2/lib/R/lib -lR -lintl -Wl,-framework -Wl,CoreFoundation


** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (glmnet)


Installing package into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/glmnet_4.1-8.tar.gz'
Content type 'application/x-gzip' length 2439515 bytes (2.3 MB)
downloaded 2.3 MB


The downloaded source packages are in
	‘/private/var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/Rtmphiu8f4/downloaded_packages’


In [11]:
! ls TSForecasting/experiments

deep_learning_experiments.py fixed_horizon_functions.R
feature_experiments.R        forecastingdata_python.ipynb
feature_functions.R          rolling_origin.R
fixed_horizon.R


In [12]:
%%R
BASE_DIR <- "TSForecasting"
source("TSForecasting/experiments/feature_functions.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


The fixed horizon we need to try line by line to skip errors

In [13]:
%%R
source("TSForecasting/experiments/fixed_horizon_functions.R")

Loading required package: greybox
Package "greybox", v2.0.0 loaded.


Attaching package: ‘greybox’

The following object is masked from ‘package:lubridate’:

    hm

The following object is masked from ‘package:tidyr’:

    spread

This is package "smooth", v4.0.0

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loaded glmnet 4.1-8


## Part 2) Using tools directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [14]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']
do_fixed_horizon_local_forecasting = robjects.r['do_fixed_horizon_local_forecasting']
do_fixed_horizon_global_forecasting = robjects.r['do_fixed_horizon_global_forecasting']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [15]:
%%capture cap --no-stderr
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "catch22")

R[write to console]: Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [16]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/sample_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/sample_features.csv

head of the tsfeatures features
sample_features.csv
mean,var,max_kl_shift,time_kl_shift,max_level_shift,time_level_shift,max_var_shift,time_var_shift,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,flat_spots,alpha,beta,hurst,lumpiness,nonlinearity,x_pacf5,diff1x_pacf5,diff2x_pacf5,seas_pacf,stability,unitroot_kpss,unitroot_pp,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough
105.652455128205,669.530932765716,1.2706668176004,102,0.755290510889678,52,0.959745807190754,52,0.710805797539225,1.51175743379058,-0.239716429411997,0.105249143633781,-0.539721534779089,0.372057773793558,-0.0890329975757608,0.257725362872141,22,0.795329846811295,14,0.628482857552256,0.000100053599224661,0.989518146632053,0.254142008742879,0.315470866286389,0.530751963124808,0.104820622256713,0.656220390192106,0.0363289043636947,0.160377933409522,0.459213533792908,-39.7197694054302,1,52,0.233768779821994,0.00

Compute the local forecasts.

In [17]:
do_fixed_horizon_local_forecasting("sample", "ses", "sample.tsf", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

<rpy2.rinterface_lib.sexp.NULLType object at 0x104080b40> [RTYPES.NILSXP]

Try the global model forecasts.

In [18]:
do_fixed_horizon_global_forecasting("sample", 25, "sample.tsf", "pooled_regression", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

<rpy2.rinterface_lib.sexp.NULLType object at 0x104080b40> [RTYPES.NILSXP]